In [12]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import pandas as pd

column_names = ['ID', 'TITLE', 'GENRE', 'DESCRIPTION']

train_data=pd.read_csv('train_data.txt',sep=' ::: ', engine='python', header=None, names=column_names)
test_data=pd.read_csv('test_data.txt',sep=' ::: ', engine='python', header=None, names=column_names)

print(train_data.head())
print(test_data.head())

   ID                             TITLE     GENRE  \
0   1      Oscar et la dame rose (2009)     drama   
1   2                      Cupid (1997)  thriller   
2   3  Young, Wild and Wonderful (1980)     adult   
3   4             The Secret Sin (1915)     drama   
4   5            The Unrecovered (2007)     drama   

                                         DESCRIPTION  
0  Listening in to a conversation between his doc...  
1  A brother and sister with a past incestuous re...  
2  As the bus empties the students for their fiel...  
3  To help their unemployed father make ends meet...  
4  The film's title refers not only to the un-rec...  
   ID                        TITLE  \
0   1         Edgar's Lunch (1998)   
1   2     La guerra de papá (1977)   
2   3  Off the Beaten Track (2010)   
3   4       Meu Amigo Hindu (2015)   
4   5            Er nu zhai (1955)   

                                               GENRE  DESCRIPTION  
0  L.R. Brane loves his life - his car, his apart...  

In [22]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.optimizers import Adam

# Load Data
column_names = ['ID', 'TITLE', 'GENRE', 'DESCRIPTION']
train_data = pd.read_csv('train_data.txt', sep=' ::: ', engine='python', header=None, names=column_names)

# Label Encoding
label_encoder = LabelEncoder()
train_data['GENRE'] = label_encoder.fit_transform(train_data['GENRE'])
num_labels = len(label_encoder.classes_)

# Tokenization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)

print("Model loaded successfully!")


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded successfully!


In [30]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

# Tokenization
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data['DESCRIPTION'])
X_train = tokenizer.texts_to_sequences(train_data['DESCRIPTION'])

# Calculate Maximum Length
# max_len = max(len(x) for x in X_train)
max_len = min(max_len, 200)  # Set a cap of 200 tokens
print(f"Maximum Sentence Length: {max_len}")


Maximum Sentence Length: 200


In [34]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
train_data['GENRE'] = label_encoder.fit_transform(train_data['GENRE'])
y_train = train_data['GENRE']

In [36]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenization
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data['DESCRIPTION'])
X_train = tokenizer.texts_to_sequences(train_data['DESCRIPTION'])

# Padding
max_len = 200
X_train = pad_sequences(X_train, maxlen=max_len)


In [38]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout

# Define GRU Model
model_gru = Sequential()
model_gru.add(Embedding(input_dim=5000, output_dim=128))
model_gru.add(GRU(128, return_sequences=True, dropout=0.2))
model_gru.add(GRU(64, dropout=0.2))
model_gru.add(Dense(64, activation='relu'))
model_gru.add(Dense(len(label_encoder.classes_), activation='softmax'))

model_gru.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print("GRU Model built successfully!")


GRU Model built successfully!


In [40]:
model_gru.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

# Evaluate
accuracy_gru = model_gru.evaluate(X_train, y_train)[1] * 100
print(f"GRU Accuracy: {accuracy_gru:.2f}%")

Epoch 1/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 180s 263ms/step - accuracy: 0.3186 - loss: 2.2833 - val_accuracy: 0.4786 - val_loss: 1.7798
Epoch 2/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 185s 273ms/step - accuracy: 0.5073 - loss: 1.6812 - val_accuracy: 0.5313 - val_loss: 1.6152
Epoch 3/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 192s 283ms/step - accuracy: 0.5668 - loss: 1.4870 - val_accuracy: 0.5513 - val_loss: 1.5443
Epoch 4/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 193s 284ms/step - accuracy: 0.6057 - loss: 1.3488 - val_accuracy: 0.5526 - val_loss: 1.5600
Epoch 5/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 194s 286ms/step - accuracy: 0.6346 - loss: 1.2491 - val_accuracy: 0.5569 - val_loss: 1.5542
Epoch 6/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 191s 282ms/step - accuracy: 0.6583 - loss: 1.1651 - val_accuracy: 0.5553 - val_loss: 1.5726
Epoch 7/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 196s 289ms/step - accuracy: 0.6795 - loss: 1.0841 - val_accuracy: 0.5516 - val_loss: 1.5978
Epoch 8/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 193s 285ms/step - accuracy: 0.7020 -

In [44]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenization
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data['DESCRIPTION'])
X_train = tokenizer.texts_to_sequences(train_data['DESCRIPTION'])

# Padding
max_len = 200
X_train = pad_sequences(X_train, maxlen=max_len)

print(f"Shape of X_train: {X_train.shape}")  # Should be (num_samples, max_len)


Shape of X_train: (54214, 200)


In [46]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Bidirectional, Dense, Dropout

model_gru = Sequential()

# Embedding Layer
model_gru.add(Embedding(input_dim=5000, output_dim=128, input_length=max_len))

# Bidirectional GRU Layer
model_gru.add(Bidirectional(GRU(128, return_sequences=True, dropout=0.2)))
model_gru.add(Bidirectional(GRU(64, dropout=0.2)))

# Dense Layers
model_gru.add(Dense(64, activation='relu'))
model_gru.add(Dense(len(label_encoder.classes_), activation='softmax'))

# Compile Model
model_gru.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print("Model successfully created!")


Model successfully created!


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [48]:
model_gru.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

# Evaluate
accuracy_hybrid = model_hybrid.evaluate(X_train, y_train)[1] * 100
print(f"Hybrid Model Accuracy: {accuracy_hybrid:.2f}%")

Epoch 1/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 263s 384ms/step - accuracy: 0.3532 - loss: 2.2350 - val_accuracy: 0.4868 - val_loss: 1.8071
Epoch 2/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 273s 403ms/step - accuracy: 0.5147 - loss: 1.7005 - val_accuracy: 0.5354 - val_loss: 1.6176
Epoch 3/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 276s 407ms/step - accuracy: 0.5743 - loss: 1.4665 - val_accuracy: 0.5516 - val_loss: 1.5435
Epoch 4/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 278s 410ms/step - accuracy: 0.6111 - loss: 1.3154 - val_accuracy: 0.5627 - val_loss: 1.5142
Epoch 5/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 289s 426ms/step - accuracy: 0.6447 - loss: 1.1979 - val_accuracy: 0.5517 - val_loss: 1.5484
Epoch 6/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 305s 449ms/step - accuracy: 0.6796 - loss: 1.0842 - val_accuracy: 0.5585 - val_loss: 1.5697
Epoch 7/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 301s 444ms/step - accuracy: 0.7045 - loss: 0.9880 - val_accuracy: 0.5529 - val_loss: 1.6665
Epoch 8/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 318s 470ms/step - accuracy: 0.7291 -

ValueError: Input 0 of layer "bidirectional" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 64)

In [42]:
from tensorflow.keras.layers import Bidirectional, Conv1D, GlobalMaxPooling1D

model_hybrid = Sequential()
model_hybrid.add(Embedding(input_dim=5000, output_dim=128, input_length=max_len))
model_hybrid.add(Conv1D(64, 5, activation='relu'))
model_hybrid.add(GlobalMaxPooling1D())
model_hybrid.add(Bidirectional(LSTM(64)))
model_hybrid.add(Dense(64, activation='relu'))
model_hybrid.add(Dense(len(label_encoder.classes_), activation='softmax'))

model_hybrid.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_hybrid.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

# Evaluate
accuracy_hybrid = model_hybrid.evaluate(X_train, y_train)[1] * 100
print(f"Hybrid Model Accuracy: {accuracy_hybrid:.2f}%")


Epoch 1/10


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


ValueError: Input 0 of layer "bidirectional" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 64)

In [32]:
from tensorflow.keras.layers import GRU, Embedding, Dense, Dropout
from tensorflow.keras.models import Sequential

model_gru = Sequential()
model_gru.add(Embedding(input_dim=5000, output_dim=128, input_length=max_len))
model_gru.add(GRU(128, return_sequences=True, dropout=0.2))
model_gru.add(GRU(64, dropout=0.2))
model_gru.add(Dense(64, activation='relu'))
model_gru.add(Dense(len(label_encoder.classes_), activation='softmax'))

model_gru.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_gru.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

# Evaluate
accuracy_gru = model_gru.evaluate(X_train, y_train)[1] * 100
print(f"GRU Accuracy: {accuracy_gru:.2f}%")


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


NameError: name 'y_train' is not defined

In [24]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.optimizers import Adam

# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))

train_encodings = tokenizer(train_data['DESCRIPTION'].tolist(), truncation=True, padding=True, max_length=256, return_tensors='tf')
test_encodings = tokenizer(test_data['DESCRIPTION'].tolist(), truncation=True, padding=True, max_length=256, return_tensors='tf')

# Compile Model
model.compile(optimizer=Adam(learning_rate=5e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_encodings['input_ids'], train_data['GENRE'], epochs=5, batch_size=16, validation_split=0.2)

# Evaluate
accuracy_bert = model.evaluate(train_encodings['input_ids'], train_data['GENRE'])[1] * 100
print(f"BERT Accuracy: {accuracy_bert:.2f}%")


ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [18]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.optimizers import Adam

# Load BERT Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize
def encode_texts(texts):
    return tokenizer(texts.tolist(), truncation=True, padding=True, max_length=200, return_tensors='tf')

train_encodings = encode_texts(train_data['DESCRIPTION'])
test_encodings = encode_texts(test_data['DESCRIPTION'])

# Model
model_bert = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))
model_bert.compile(optimizer=Adam(learning_rate=5e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train
history = model_bert.fit(train_encodings['input_ids'], train_data['GENRE'], epochs=3, batch_size=16, validation_split=0.2)

# Evaluation
accuracy_bert = model_bert.evaluate(train_encodings['input_ids'], train_data['GENRE'])[1] * 100
print(f"BERT Model Training Accuracy: {accuracy_bert:.2f}%")


ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).